# Exercise 1.

Although merge sort has a better Big-O than selection sort, selection sort can be faster for smaller inputs.

Rewrite `merge_sort(A, min_size)` such that sub-arrays smaller than an input parameter `min_size` are sorted with our `selection_sort` from the lecture `algorithms intro`.

Time the difference between pure merge sort and this new algorithm. Is it faster? Why or why not?

In [37]:
import numpy as np

def linear_search(arr):
    """
    Find the index of the minimum element
    AKA argsort
    """
  # initialize current best to +infinity
  # So any element beats it
    current_min = float('inf')
    current_min_idx = 0
    for i in range(len(arr)):
        if arr[i] < current_min:
            current_min = arr[i]
            current_min_idx = i
    return current_min_idx


def selection_sort(arr):
    """Selection sort"""
    n_sorted = 0
    while n_sorted < len(arr):
    # Get the index of the min of remaining elements
    # Since argsort returns based on array, we correct result
    # with `+ n_sorted`
        min_idx = linear_search(arr[n_sorted:]) + n_sorted
    # Swap minimum element with leftmost remaining element
        to_swap = arr[n_sorted]
        arr[n_sorted] = arr[min_idx]
        arr[min_idx] = to_swap
    # Increment and restart
        n_sorted += 1
        

def merge(left, right):
    res = []
    #zip through R and L parts
    while len(left) > 0 and len(right) > 0:
        if left[0] < right[0]:
            res.append(left[0])
            left.pop(0)
        else:
            res.append(right[0])
            right.pop(0)
    for i in left:
        res.append(i)
    for i in right:
        res.append(i)
    return res


def merge_sort_pure(arr):
    size = len(arr)
    if size > 1:
        mid = size // 2
        left = merge_sort_pure(arr[:mid])
        right = merge_sort_pute(arr[mid:])
        return merge(left,right)
    else:
        return arr

arr = [111,4,3,22,5,44.4,66.6,777]
selection_sort(arr)
arr

[3, 4, 5, 22, 44.4, 66.6, 111, 777]

In [54]:
def merge_sort(arr, min_size=0):
    
    if len(arr) <= 1: #if array is >= 1 just return array. Nothing to sort
        return arr
    elif len(arr) < min_size: #if less than min_size, use selection_sort
        return selection_sort(arr)
    else:
        midpt = len(arr)//2 #find midpoint to divide array by 2 & plug 2 part array into merge func
        return merge(merge_sort(arr[:midpt]),merge_sort(arr[midpt:])) 

a = [2,5,1,3,4,1.5,2.5]
%timeit merge_sort(a,)
%timeit merge_sort(a,20)

#using the new algorithm which incorporates search_sort fn is slightly quicker than using
#only the pure merge_sort without search_sort

6.61 µs ± 205 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
5.6 µs ± 352 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


# Exercise 2. 

Let $A[1...n]$ be an array of $n$ distinct numbers. If $i < j$ and $A[i] > A[j]$, then the pair $(i, j)$ is called an **inversion** of $A$. 

In other words an inversion is a pair of unsorted elements in an array.

**1)** List the five inversions of $[2, 3, 8, 6, 1]$ 

**2)** Give an algorithm that determines the number of inversions in any permutation on $n$ elements in $O(nlog_2(n))$ worst-case time. (Hint: Modify merge sort.)

In [88]:
#Number of inversions can be calculated as n(n-1) / 2
# 1: List the five inversions:

def num_inversions(arr): #for recursive fn (below)
    """
    Counts the number of inversions
    in a selected array
    """
    inv_count = 0
    for i in range(len(arr)): #i in range or arr 
        for j in range(i+1, len(arr)): #j is i + 1 because we are starting one index over and ending in the len of arr
            if arr[i] > arr[j]: #see if arr[i] > arr[j] and if so, add to inv_count
                inv_count += 1
    return inv_count #return number of inversions

def list_inversions(arr):
    """
    Lists the inversions in
    a specified array
    """
    inv = []
    
    for i in range(len(arr)):
        for j in range(i+1, len(arr)):
            if arr[i] > arr[j]:
                inv.append((arr[i],arr[j]))
    return inv

print(f"The array has {num_inversions([2,3,8,6,1])} inversions")
print(f"The inversion pairs are: {list_inversions([2,3,8,6,1])}")

The array has 5 inversions
The inversion pairs are: [(2, 1), (3, 1), (8, 6), (8, 1), (6, 1)]


In [5]:
# got a whole lot of help: https://www.geeksforgeeks.org/counting-inversions/
#Recursive inversion count:

def inv_merge_sort(arr):
    """
    Function to use inversion count.
    Temp_arr is created to store
    sorted array in merge function
    """
    temp_arr = [0]*len(arr) #temp array of 0's the size of orig array
    return count_inv_merge_sort(arr,temp_arr,0,(len(arr))-1)

def count_inv_merge_sort(arr,temp_arr,left,right):
    """
    Will use inv_merge_sort fn to count
    number of inversions in array.
    """
    inv_count = 0 #used to store inv counts in each recursive call
    
    if left < right:
        mid = (left + right) // 2
        #Count inversion in left subarray
        inv_count += count_inv_merge_sort(arr,temp_arr,left,mid)
        #Count inversion in right subarray
        inv_count += count_inv_merge_sort(arr,temp_arr,mid+1,right)
        #sort and merge subarrays
        inv_count += inv_merge(arr,temp_arr,left,mid,right)
    return inv_count

def inv_merge(arr,temp_arr,left,mid,right):
    """
    Merge two subarrays into a single
    sorted array
    """
    i = left    #Start index of left subarray
    j = mid + 1 #Start index of right subarry
    k = left    #Start index of to be sorted subarray
    inv_count = 0
    
    #Ensures that i and j don't exceet their subarray limits    
    while i <= mid and j <= right:
        if arr[i] <= arr[j]: #no inversions if i <= j
            temp_arr[k] = arr[i] #plugging in to temp array with left index 
            k += 1
            i += 1
        else:                #inversion if i > j
            temp_arr[k] = arr[j]
            inv_count += (mid-i + 1)
            k += 1
            j += 1

    while i <= mid: #copy remaining elems of the left subrray into temp arr
        temp_arr[k] = arr[i]
        k += 1
        i += 1
        
    while j <= right: #copy rem elems of the right subarray into temp arr
        temp_arr[k] = arr[j]
        k += 1
        j += 1
        
    #copy sroted subarray into original array
    for _ in range(left,right+1):
        arr[_] = temp_arr[_]
        
    return inv_count

arr = [2,3,8,6,1]
arr2 = [4,6,1,3,2]
print(f"There are {inv_merge_sort(arr)} inversions in [2,3,8,6,1]")
print(f"There are {inv_merge_sort(arr2)} inversions in [4,6,1,3,2]")

There are 5 inversions in [2,3,8,6,1]
There are 7 inversions in [4,6,1,3,2]


# 3. Recursive sum

Write a function that uses recursion to compute the sum of an array or list of numbers

```
recursive_sum([2, 4, 5, 6, 7])

output: 24
```

In [38]:
import numpy as np
def recursive_sum(arr):
    """
    Get the sum of a list
    or array using recursion
    """
    if len(arr) == 0: #if arr has no elements, return 0
        return 0
    else:
        return arr[0] + recursive_sum(arr[1:]) #selse return sum starting with arr[0] 
                                               #a[0] or 1 + a[1] or 2 = 3 *notes for self*
                                               #3 becomes a[0]...a[0] or 3 + a[1] or 3 = 6
                                               #6 becomes a[0]...a[0] or 6 + a[1] or 4 = 10
    

x = [1,2,3,4]
x = np.array(x)
x
y = [1,2,3,4]
print(recursive_sum(x))
print(recursive_sum(y))

10
10


# 4. Recursive denominators

Write a Python program that uses recursion to find the greatest common divisor (gcd) of two integers.

```
recursive_gcd(12,14)

output : 2
```

In [39]:
def recursive_gcd(a,b):
    """
    Finds greatest common divisor
    for two integers using recursion
    """
    if b == 0: #if be is 0, return a. If a is 0, b is returned in else statement.
        return a 
    else:
        return gcd_recursive(b, a % b)# switches it up from my understanding
                                      # if modulo returns the same a val, then it goes back and switches
                                      # numbers and gets modulo of other values
    
recursive_gcd(18,24)

6

# 5. Recursive power function

Write a recursive function to calculate the value of 'a' to the power 'b'. 

```
recursive_pow(3, 4)

output: 81
```

In [97]:
def recursive_pow(a,b):
    """
    Calculate the value 'a'to the
    power of 'b' using recursion
    """
    if b == 0:
        return 1 #if power is 0, return 1
    else:
        return a * recursive_pow(a, b - 1) 
    #else multiply a given number to itself until the power reaches 0
    #rec_pow(2,3): 
    #2 * rec_pow(2,3-1 or 2) = 2 * 2 * rec_pow(2,1) = 2 * 2 * 2 * rec_pow(2,0) = 2 * 2 * 2 * 1 (b/c b==0) = 8
    
    
recursive_pow(5,5)

3125

# 6. (Stretch) K-Nearest Neighbours

Consider a matrix with the following format:

```
[[0.3, 0.8],
 [-0.2, 0.5],
 [1, -1],
 [0.9, 0.5]
]
```

Each row denotes a point, and the numbers in each row are the coordinates. The coordinates in this example are in 2d, but the matrix could be in 3d (3 numbers per row) or even higher dimensions.

Your task is to write a function `knn(m, p)` or `k_nearest_neighbors(m, p, k)` which takes in a matrix of points `m`, an integer `p` denoting the index of a point in that matrix, and an intger `k` denoting the number of nearest neighbors to return.

The function returns the index of the `k` nearest neighbors of the point `p` in the matrix `m`.

```
dataset = [[2.7810836,2.550537003,0],
	[1.465489372,2.362125076,0],
	[3.396561688,4.400293529,0],
	[1.38807019,1.850220317,0],
	[3.06407232,3.005305973,0],
	[7.627531214,2.759262235,1],
	[5.332441248,2.088626775,1],
	[6.922596716,1.77106367,1],
	[8.675418651,-0.242068655,1],
	[7.673756466,3.508563011,1]]

knn(dataset, 0, 2)

output : [4, 1]
```

You can use `from sklearn.neighbors import NearestNeighbors` to test your function